In [1]:
import mysql.connector
from tabulate import tabulate

In [2]:
cnx = mysql.connector.connect(user='root',
                              password='root',
                              host='mysql',
                              port='3306',
                              db='country_club')
cursor = cnx.cursor()

#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [3]:
cursor.execute(
    """
    SELECT name 
    FROM Facilities
    WHERE membercost > 0
    """
)

for (facility,) in cursor:
    print(facility)

Tennis Court 1
Tennis Court 2
Massage Room 1
Massage Room 2
Squash Court


#### Q2: How many facilities do not charge a fee to members?

In [4]:
cursor.execute(
    """
    SELECT COUNT(*) 
    FROM Facilities
    WHERE membercost = 0
    """
)

for (free_count,) in cursor:
    print(free_count)

4


#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [5]:
cursor.execute(
    """
    SELECT facid, name, membercost, monthlymaintenance
    FROM Facilities
    WHERE (membercost / monthlymaintenance < 0.2) & (membercost > 0)
    """
)

data = cursor.fetchall()
column_names = cursor.column_names
print(tabulate(data, headers=column_names))

  facid  name              membercost    monthlymaintenance
-------  --------------  ------------  --------------------
      0  Tennis Court 1           5                     200
      1  Tennis Court 2           5                     200
      4  Massage Room 1           9.9                  3000
      5  Massage Room 2           9.9                  3000
      6  Squash Court             3.5                    80


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [6]:
cursor.execute(
    """
    SELECT *
    FROM Facilities
    WHERE facid IN (1, 5)
    """
)

data = cursor.fetchall()
column_names = cursor.column_names
print(tabulate(data, headers=column_names))

  facid  name              membercost    guestcost    initialoutlay    monthlymaintenance
-------  --------------  ------------  -----------  ---------------  --------------------
      1  Tennis Court 2           5             25             8000                   200
      5  Massage Room 2           9.9           80             4000                  3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than \$100? Return the name and monthly maintenance of the facilities in question.

In [7]:
cursor.execute(
    """
    SELECT name, monthlymaintenance,
        CASE 
            WHEN monthlymaintenance > 100 
            THEN 'expensive'
            ELSE 'cheap'
        END AS cost
    FROM Facilities;
    """
)

data = cursor.fetchall()
column_names = cursor.column_names
print(tabulate(data, headers=column_names))

name               monthlymaintenance  cost
---------------  --------------------  ---------
Tennis Court 1                    200  expensive
Tennis Court 2                    200  expensive
Badminton Court                    50  cheap
Table Tennis                       10  cheap
Massage Room 1                   3000  expensive
Massage Room 2                   3000  expensive
Squash Court                       80  cheap
Snooker Table                      15  cheap
Pool Table                         15  cheap


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.


In [8]:
cursor.execute(
    """
    SELECT a.firstname, a.surname
        FROM (SELECT firstname, surname, MAX(joindate)
             FROM Members 
             WHERE surname != 'GUEST') a
    """
)

data = cursor.fetchall()
column_names = cursor.column_names
print(tabulate(data, headers=column_names))

firstname    surname
-----------  ---------
Darren       Smith


#### Q7: How can you produce a list of all members who have used a tennis court? Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name.

In [9]:
cursor.execute(
    """
    SELECT DISTINCT CONCAT(m.firstname, ' ', m.surname) AS member
    FROM Members m
    JOIN (SELECT b.facid, b.memid FROM Bookings b
          JOIN Facilities f
          ON b.facid = f.facid AND f.name LIKE 'Tennis Court%') a
    ON m.memid = a.memid AND m.memid != 0
    ORDER BY member
    """
)

data = cursor.fetchall()
column_names = cursor.column_names
print(tabulate(data, headers=column_names))

member
-----------------
Anne Baker
Burton Tracy
Charles Owen
Darren Smith
David Farrell
David Jones
David Pinker
Douglas Jones
Erica Crumpet
Florence Bader
Gerald Butters
Henrietta Rumney
Jack Smith
Janice Joplette
Jemima Farrell
Joan Coplin
John Hunt
Matthew Genting
Millicent Purview
Nancy Dare
Ponder Stibbons
Ramnaresh Sarwin
Tim Boothe
Tim Rownam
Timothy Baker
Tracy Smith


#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30. Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.

In [10]:
cursor.execute(
    """
    SELECT f.name AS facilityname,
        CASE WHEN m.firstname = 'GUEST'
             THEN m.firstname
             ELSE CONCAT(m.firstname,  ' ', m.surname)
        END AS membername,
        CASE WHEN b.memid = 0 
             THEN f.guestcost * b.slots
             ELSE f.membercost * slots
        END AS cost
        FROM Bookings b
        JOIN Members m ON b.memid = m.memid AND DATE(b.starttime) = '2012-09-14'
        JOIN Facilities f ON b.facid = f.facid HAVING cost > 30 
        ORDER BY cost DESC
    """
)
    
data = cursor.fetchall()
column_names = cursor.column_names
print(tabulate(data, headers=column_names))

facilityname    membername        cost
--------------  --------------  ------
Massage Room 2  GUEST            320
Massage Room 1  GUEST            160
Massage Room 1  GUEST            160
Massage Room 1  GUEST            160
Tennis Court 2  GUEST            150
Tennis Court 2  GUEST             75
Tennis Court 1  GUEST             75
Tennis Court 1  GUEST             75
Squash Court    GUEST             70
Massage Room 1  Jemima Farrell    39.6
Squash Court    GUEST             35
Squash Court    GUEST             35


#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [11]:
cursor.execute(
    """
    SELECT a.facilityname, a.membername, a.cost
    FROM (SELECT f.name AS facilityname,
              CASE WHEN m.firstname = 'GUEST'
                   THEN m.firstname
                   ELSE CONCAT(m.firstname,  ' ', m.surname)
              END AS membername,
              CASE WHEN b.memid = 0 
                   THEN f.guestcost * b.slots
                   ELSE f.membercost * slots 
              END AS cost         
          FROM country_club.Bookings b
          JOIN Members m ON b.memid = m.memid AND b.starttime BETWEEN '2012-09-14' AND '2012-09-15'
          JOIN Facilities f ON b.facid = f.facid) a
    WHERE a.cost > 30
    ORDER BY cost DESC
    """
)

data = cursor.fetchall()
column_names = cursor.column_names
print(tabulate(data, headers=column_names))

facilityname    membername        cost
--------------  --------------  ------
Massage Room 2  GUEST            320
Massage Room 1  GUEST            160
Massage Room 1  GUEST            160
Massage Room 1  GUEST            160
Tennis Court 2  GUEST            150
Tennis Court 2  GUEST             75
Tennis Court 1  GUEST             75
Tennis Court 1  GUEST             75
Squash Court    GUEST             70
Massage Room 1  Jemima Farrell    39.6
Squash Court    GUEST             35
Squash Court    GUEST             35


#### Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!
    

In [12]:
cursor.execute(
    """
    SELECT f.name AS facilityname, 
           SUM(CASE 
                   WHEN b.memid = 0 
                   THEN f.guestcost * b.slots
                   ELSE f.membercost * b.slots 
               END)
           AS totalrevenue
    FROM Facilities f
    JOIN Bookings b ON f.facid = b.facid
    GROUP BY f.name HAVING totalrevenue <1000 
    ORDER BY totalrevenue
    """
)
data = cursor.fetchall()
column_names = cursor.column_names
print(tabulate(data, headers=column_names))

facilityname      totalrevenue
--------------  --------------
Table Tennis               180
Snooker Table              240
Pool Table                 270


In [13]:
cnx.close()
